In [2]:
# Cài đặt thư viện cần thiết
!pip install transformers datasets evaluate

# Import thư viện
from transformers import pipeline
from datasets import load_dataset
import evaluate

# Load mô hình ZycckZ/VieModTest
qa_pipeline = pipeline("question-answering", model="ZycckZ/VieModTest", tokenizer="ZycckZ/VieModTest")

# Load bộ dữ liệu đánh giá (ví dụ: UIT-ViQuAD2.0)
dataset = load_dataset("taidng/UIT-ViQuAD2.0", split="validation")

# Load metrics
squad_metric = evaluate.load("squad")

# Hàm chạy đánh giá
def evaluate_model(dataset, qa_pipeline):
    predictions = []
    references = []

    for item in dataset:
        context = item["context"]
        question = item["question"]

        # Check if the 'text' list is empty before accessing it
        # If empty, skip this instance
        if not item["answers"]["text"]:
            continue

        ground_truth = item["answers"]["text"][0]  # Lấy câu trả lời đúng đầu tiên


        pred = qa_pipeline(question=question, context=context)

        predictions.append({
            "id": item["id"],
            "prediction_text": pred["answer"]
        })
        references.append({
            "id": item["id"],
            "answers": {
                "answer_start": item["answers"]["answer_start"],
                "text": item["answers"]["text"]
            }
        })

    # Tính điểm
    results = squad_metric.compute(predictions=predictions, references=references)
    return results

# Chạy đánh giá
results = evaluate_model(dataset, qa_pipeline)

# In kết quả
print("Đánh giá mô hình ZycckZ/VieModTest:")
print(f"Exact Match (EM): {results['exact_match']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")


Device set to use cuda:0


Đánh giá mô hình ZycckZ/VieModTest:
Exact Match (EM): 50.28
F1 Score: 70.75
